In [2]:
using JuMP
using CSV
using LinearAlgebra
using DataFrames
using BARON

In [3]:
A_db = CSV.File("./A_mat.csv",header=1,delim=",") |> DataFrame
B_db = CSV.File("./B_mat.csv",header=1,delim=",") |> DataFrame
f_db = CSV.File("./f_val.csv",header=1,delim=",") |> DataFrame
f_val=f_db[!,2]
A_mat=Matrix(A_db[:,2:size(A_db,2)])
B_mat=Matrix(B_db[:,2:size(B_db,2)])
nodes_val=names(A_db)[2:45]
product_val=A_db[!,1]

52-element Vector{String63}:
 "Methanol "
 "Ethylene "
 "Propylene "
 "Toluene "
 "Benzene "
 "Xylene "
 "Butene "
 "Propane "
 "Syngas"
 "Urea "
 "Ammonia "
 "Acrylonitrile"
 "Cyclohexane "
 ⋮
 "Adipic Acid"
 "Nylon 6 6 resin"
 "Polyurethanes"
 "Adipate ester"
 "Packaging"
 "Consumer Goods/Lifestyle"
 "Healthcare & Pharmaceuticals"
 "Electrical/Electronics"
 "Bottles"
 "Sheet/Film"
 "others"
 "Gasoline Additives"

In [4]:
B_db

,Column1,Syngas Production,Methanol Manufacture,Steam Cracker,Hydrodealkylation (THD)
,String31,Float64,Float64,Float64,Float64
1,Coal,-83.5323,-38.1901,0.0,0.0
2,Natural Gas,-79.342,-36.0639,-94.3427,0.0
3,Waste,31.0763,18.2015,23.464,0.377508
4,Environment,47.6829,19.8492,0.0,1.41346
5,Oil,0.0,-9.44493,-362.702,0.0
6,Coal Energy,-89.1436,-27.9993,-131.271,-0.876937
7,NG Energy,-47.6952,-14.9807,0.0,-0.469194
8,Environment Energy,136.839,42.9799,131.271,1.34613


In [35]:
function opt_decarb()
    dcrb =  Model(BARON.Optimizer)
    @variable(dcrb,s[j=1:size(A_mat,2)] ≥ 0)
    @variable(dcrb,f[i=1:size(A_mat,1)] ≥ 0 )
    @variable(dcrb,g[k=1:size(B_mat,1)])
    @constraint(dcrb,[i=1:size(A_mat,1)],f[i] ≥ f_val[i]/2)
    @constraint(dcrb,[i=1:size(A_mat,1)], f[i]==sum(A_mat[i,j]*s[j] for j in 1:size(A_mat,2)))
    @constraint(dcrb,[k=1:size(B_mat,1)],g[k]==sum(B_mat[k,j]*s[j] for j in 1:size(B_mat,2)))
    @variable(dcrb,obj)
    @constraint(dcrb,obj==sum(g[k] for k in 3:4)+g[8])
    @objective(dcrb,Min,obj)
    optimize!(dcrb)
    return JuMP.value.(s),JuMP.value.(f),JuMP.value.(obj)
end
    

opt_decarb (generic function with 1 method)

In [36]:
s,f,obj=opt_decarb()

 BARON version 21.1.13. Built: LNX-64 Wed Jan 13 16:09:31 EST 2021

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Bhavik R. Bakshi at The Ohio State University, bakshi.2@osu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSD, FILTERSQP
 Solving bounding LP
 Preprocessing found feasible solution with value  1007.55300746     
 Problem solved during preprocessing
 Lower bound is  1007.55300746     

 Calculating duals

                         *** Normal completion ***            

 Wall clock time:                     0.00
 Total CPU time used:             

([1.000000000048745, 1.000000000008715, 1.0090556646623505, 0.936214417575971, 1.069578463617712, 0.0, 1.9580893230052303, 0.0, 1.9580893230052308, 1.0  …  1.0, 1.0000000000787401, 1.0, 1.0000000000000002, 1.0, 0.9999999999632565, 1.0000000002140417, 1.0, 0.9999999996881713, 1.0], [38.836527069500015, 33.9216148645, 1.131189476999995, 64.33685002, 15.8587545845, 19.996879946795097, 0.0, 18.03485710456751, 0.0, 65.4  …  0.48082191800000007, 0.24041096000000006, 87.84299584855137, 26.473373702510276, 5.142857144, 10.758058254952063, 44.65681674, 8.805569499, 0.48082191800000007, 313.6797617], 1007.5530074631616)

In [37]:
obj

1007.5530074631616

In [38]:
res_f=DataFrame(pdt=product_val, final_demand=f)
res_s=DataFrame(node=nodes_val, scaling_factor=s)

,node,scaling_factor
,String,Float64
1,Syngas Production,1.0
2,Methanol Manufacture,1.0
3,Steam Cracker,1.00906
4,Hydrodealkylation (THD),0.936214
5,Disproportionation (TDP),1.06958
6,Combustion,0.0
7,Metathesis,1.95809
8,Dehydrogenation,0.0
9,Refinery,1.95809


In [39]:
res_f

,pdt,final_demand
,String63,Float64
1,Methanol,38.8365
2,Ethylene,33.9216
3,Propylene,1.13119
4,Toluene,64.3369
5,Benzene,15.8588
6,Xylene,19.9969
7,Butene,0.0
8,Propane,18.0349
9,Syngas,0.0


In [40]:
B_db

,Column1,Syngas Production,Methanol Manufacture,Steam Cracker,Hydrodealkylation (THD)
,String31,Float64,Float64,Float64,Float64
1,Coal,-83.5323,-38.1901,0.0,0.0
2,Natural Gas,-79.342,-36.0639,-94.3427,0.0
3,Waste,31.0763,18.2015,23.464,0.377508
4,Environment,47.6829,19.8492,0.0,1.41346
5,Oil,0.0,-9.44493,-362.702,0.0
6,Coal Energy,-89.1436,-27.9993,-131.271,-0.876937
7,NG Energy,-47.6952,-14.9807,0.0,-0.469194
8,Environment Energy,136.839,42.9799,131.271,1.34613


In [42]:
f_db

,Product,Final Demands
,String63,Float64
1,Methanol,77.6731
2,Ethylene,67.8432
3,Propylene,2.26238
4,Toluene,128.674
5,Benzene,31.7175
6,Xylene,38.8426
7,Butene,0.0
8,Propane,0.0
9,Syngas,-3.99999e-9
